In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
x_train = newsgroups_train.data
x_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target
print ("20개 카테고리 전체 목록:")
print (newsgroups_train.target_names)
print ("\n")
print ("샘플 이메일:")
print (x_train[0])
print ("샘플 타깃 카테고리:")
print (y_train[0])
print (newsgroups_train.target_names[y_train[0]])

List of all 20 categories:
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Sample Email:
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is

In [3]:
# 데이터 전처리에 사용 Used for pre-processing data
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer


In [5]:
def preprocessing(text):
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())

    tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
    
    tokens = [word.lower() for word in tokens]
    
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    
    tokens = [word for word in tokens if len(word)>=3]
    
    stemmer = PorterStemmer()
    try:
        tokens = [stemmer.stem(word) for word in tokens]

    except:
        tokens = tokens
        
    tagged_corpus = pos_tag(tokens)    
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']

    lemmatizer = WordNetLemmatizer()

    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    
    pre_proc_text =  " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])             

    return pre_proc_text


In [6]:
x_train_preprocessed  = []
for i in x_train:
	x_train_preprocessed .append(preprocessing(i))

x_test_preprocessed = []
for i in x_test:
	x_test_preprocessed.append(preprocessing(i))


In [7]:
# TFIDF 벡터라이저vectorizer 구축
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2),  stop_words='english', 
                             max_features= 10000,strip_accents='unicode',  norm='l2')

x_train_2 = vectorizer.fit_transform(x_train_preprocessed).todense()
x_test_2 = vectorizer.transform(x_test_preprocessed).todense()

In [10]:
# 딥러닝 모듈
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


In [11]:
# 하이퍼 파라미터 정의
np.random.seed(1337) 
nb_classes = 20
batch_size = 64
nb_epochs = 20

Y_train = np_utils.to_categorical(y_train, nb_classes)


In [12]:
#Deep Layer Model building in Keras
#del model
# 케라스의 딥 레이어(심층) 모델 구축
#del 모델

model = Sequential()

model.add(Dense(1000,input_shape= (10000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              10001000  
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                25050     
__________

In [13]:
# 모델 학습
model.fit(x_train_2, Y_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

Epoch 1/20
11314/11314 [==============================] - 112s 10ms/step - loss: 1.9580
Epoch 2/20
11314/11314 [==============================] - 82s 7ms/step - loss: 0.5893
Epoch 3/20
11314/11314 [==============================] - 79s 7ms/step - loss: 0.3023
Epoch 4/20
11314/11314 [==============================] - 80s 7ms/step - loss: 0.1762
Epoch 5/20
11314/11314 [==============================] - 79s 7ms/step - loss: 0.1268
Epoch 6/20
11314/11314 [==============================] - 77s 7ms/step - loss: 0.0811
Epoch 7/20
11314/11314 [==============================] - 76s 7ms/step - loss: 0.0728
Epoch 8/20
11314/11314 [==============================] - 77s 7ms/step - loss: 0.0653
Epoch 9/20
11314/11314 [==============================] - 96s 9ms/step - loss: 0.0476
Epoch 10/20
11314/11314 [==============================] - 79s 7ms/step - loss: 0.0441
Epoch 11/20
11314/11314 [==============================] - 77s 7ms/step - loss: 0.0373
Epoch 12/20
11314/11314 [=========================

In [14]:
# 모델 예측
y_train_predclass = model.predict_classes(x_train_2,batch_size=batch_size)
y_test_predclass = model.predict_classes(x_test_2,batch_size=batch_size)

from sklearn.metrics import accuracy_score,classification_report

print ("\n\n딥 뉴럴 네트워크  - 학습 정확도:"),(round(accuracy_score(y_train,y_train_predclass),3))
print ("\n딥 뉴럴 네트워크  - 테스트 정확도:"),(round(accuracy_score(y_test,y_test_predclass),3))

print ("\n딥 뉴럴 네트워크  - 학습 분류 리포트Train Classification Report")
print (classification_report(y_train,y_train_predclass))

print ("\n딥 뉴럴 네트워크  - 테스트 분류 리포트Test Classification Report")
print (classification_report(y_test,y_test_predclass))



딥 뉴럴 네트워크  - 학습 정확도:

딥 뉴럴 네트워크  - 테스트 정확도:

딥 뉴럴 네트워크  - 학습 분류 리포트Train Classification Report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       480
          1       1.00      1.00      1.00       584
          2       1.00      1.00      1.00       591
          3       1.00      1.00      1.00       590
          4       1.00      1.00      1.00       578
          5       1.00      1.00      1.00       593
          6       0.99      1.00      1.00       585
          7       1.00      1.00      1.00       594
          8       1.00      1.00      1.00       598
          9       1.00      1.00      1.00       597
         10       1.00      1.00      1.00       600
         11       1.00      1.00      1.00       595
         12       1.00      0.99      1.00       591
         13       1.00      1.00      1.00       594
         14       1.00      1.00      1.00       593
         15       1.00      1.00      1.00       599
 